In [ ]:
 ! pip install pymc3
 #! conda install python-graphviz

In [ ]:
!pip install -U pymc3[plots] -q

In [ ]:
# Imports
import pymc3 as pm
import numpy.random as npr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import Counter
import seaborn as sns

# Set plotting style
sns.set_style('white')
sns.set_context('poster')

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')

# Bayesian Inference with PyMC3

The notebook borrows examples from the following sources: 
- [**ericmjl**.github.io/**bayesian-stats-talk**](https://ericmjl.github.io/bayesian-stats-talk)
- PyMC3 tutorials https://docs.pymc.io/nb_tutorials/index.html

## Bayesian inference in one equation

$$ P(\theta|D) = \frac{P(D|\theta)P(\theta)}{P(D)} $$

- $ P(D|\theta) $: Probability of the data arising given the parameters (Likelihood).
- $ P(\theta|D) $: Distribution of the parameters given the data (Posterior).
- $ P(\theta) $: Prior belief about the parameter (Prior).
- $ P(D) $: Probability of the data arising, globally (Difficult to compute).

## Practical reasons for bayesian inference

- can incorporate prior knowledge and belief   
- can update models after new data becomes available  
- provide a way to fit complicated models  


## `pymc3`

- Some simple Bayesian models can be computed by hand. See readings: _The Conjugate Prior for the Normal Distribution_.
- Most Bayesian models are difficult to solve because of **complicated integrals** needed to compute **posterior distributions** (the $P(D)$ part).
- **M**arkov **C**hain **M**onte **C**arlo (MCMC) sampling enables us to **estimate shape of posterior distributions** using simulation; calculus not required. See readings: _MCMC sampling for dummies_


![](https://github.com/pymc-devs/pymc3/blob/master/docs/pymc3_logo.jpg?raw=true)

- Library of **statistical distributions**, **sampling algorithms**, and **syntax** for specifying statistical models
- Everything in Python!
- Alternatives are [stan](https://mc-stan.org/), [Edward](http://edwardlib.org), and [PyRo](https://pyro.ai/).

## parameter estimation

Frequetist: "is the true value equal to X?"

OR

Bayesian: "given the data, for the parameter of interest, what is the probability distribution over the possible values?"

# Example 1: Coin toss problem

I tossed my coin $ n $ times, and it came up as heads $ h $ times. Is it biased?

In our example, assume that the data is $n = 30$, and $h = 12$

## Questions to ask
1. Bayesian: "I want to know the distribution of $ p $, the probability of tossing heads. Given $ n $ tosses and $ h $ observed heads."

Note that this question can not be asked in a frequentist framework. As a frequentist, $p$ is a fixed number, and the question would be "What is the $p$ that is most likely to have generated the sample?"


2. Bayesian: "What is the probaility that the coin is unbiased for practical purposes, i.e.,
 $ p $ in the interval $[0.48, 0.52]$?"

Note that this question can not be asked in a frequentist framework. A frequentist can only ask: "assuming the null hypothesis $p = 0.5$ is true, how likely do I observe such a sample (p-value)?"

## Prior

- prior belief about parameter: $ p \sim Uniform(0, 1) $
- likelihood function: $ data \sim Bernoulli(p) $

## Data

In [ ]:
tosses = [1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0] #12 heads out of 30 tosses

In [ ]:
def plot_coins():
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.bar(list(Counter(tosses).keys()), list(Counter(tosses).values()))
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['tails', 'heads'])
    ax.set_ylim(0, 20)
    ax.set_yticks(np.arange(0, 21, 5))
    return fig

In [ ]:
fig = plot_coins()
plt.show()    

**What is the MLE for $p$?**

## Model (how we think the data is generated)

In [ ]:
# Context manager syntax. `coin_model` is **just** 
# a placeholder
with pm.Model() as coin_model: 
    # Distributions are PyMC3 objects.
    # Specify prior using Uniform object.
    p_prior = pm.Uniform('p', 0, 1)  
    # Specify likelihood using Bernoulli object.
    like = pm.Bernoulli('toss', p=p_prior, 
                        observed=tosses)  
                        # "observed=data" is key
                        # for likelihood.

In [ ]:
# do not worry about it in Colab
pm.model_to_graphviz(coin_model)

The above graph uses the [Plate Notation](https://en.wikipedia.org/wiki/Plate_notation) to represent a Bayesian graphical model. 

Our prior is not really informative. 

In [ ]:
from scipy.stats import norm, uniform
x= np.arange(-1,2,0.01)
plt.plot(x, uniform.pdf(x))
plt.show()

## MCMC Inference 

In [ ]:
with coin_model:
    step = pm.Metropolis()
    # sample from the posterior distribution
    coin_trace = pm.sample(5000, step=step)

## Results

In [ ]:
pm.traceplot(coin_trace, combined=True)
plt.show()

In [ ]:
coin_trace.get_values('p')

The 95% **[Credible Interval](https://en.wikipedia.org/wiki/Credible_interval)** (not confidence interval!!) is: 

In [ ]:
np.percentile(coin_trace.get_values('p'), [2.5, 97.5])

**Bayesian alternative to Hypothesis Testing:**  
H0: The coin is balanced (p = 0.5)

In [ ]:
pm.plot_posterior(coin_trace,
                  rope=[0.47, 0.53], point_estimate='mean', 
                  ref_val=0.5, hdi_prob=0.95)
plt.show()

- We can compare the 95% highest posterior density (HPD) (or credible interval) with region of practical equivalence (ROPE) as an equivalence to hypothesis testing in frequentist inference.  (See [this article](http://doingbayesiandataanalysis.blogspot.com/2013/08/how-much-of-bayesian-posterior.html) )
- In our example, <font style="color:black; font-weight:bold">95% highest posterior density (HPD)</font> encompasses the <font style="color:red; font-weight:bold">region of practical equivalence (ROPE)</font>. 
- Cannot decide for certain if the coin is fair, need to get more data or change our prior!

## What if the prior is different?
Say the treasury of the United States handed you a newly minted coin, and your prior belief is that the coin is more likely to be balanced. Our prior is very informative.  

In [ ]:
x= np.arange(0,1,0.01)
plt.plot(x, norm.pdf(x, 0.5, 0.001))
plt.show()

In [ ]:
with pm.Model() as coin_model: 
    # a very strong prior
    p_prior = pm.Normal('p', 0.5, 0.01)  
    like = pm.Bernoulli('toss', p=p_prior, 
                        observed=tosses)  
            
with coin_model:
    step = pm.Metropolis()
    coin_trace = pm.sample(5000, step=step)


In [ ]:
            
pm.plot_posterior(coin_trace[100:], color='#87ceeb', 
                  rope=[0.47, 0.53], point_estimate='mean', 
                  ref_val=0.5, hdi_prob=0.95)
plt.show()

## Bimodal prior

In [ ]:
from scipy.stats import beta

x= np.arange(0,1,0.01)
plt.plot(x, beta.pdf(x, 0.3, 0.6))
plt.show()

In [ ]:
with pm.Model() as coin_model: 
    # a bimodal
    p_prior = pm.Beta('p', 0.5, 0.5)  
    like = pm.Bernoulli('toss', p=p_prior, 
                        observed=tosses)  
            
with coin_model:
    step = pm.Metropolis()
    coin_trace = pm.sample(5000, step=step)

In [ ]:
pm.plot_posterior(coin_trace[100:], color='#87ceeb', 
                  point_estimate='mean', 
                  ref_val=0.5, hdi_prob=0.95)
plt.show()

##  Summary

1. parameterize your problem using statistical distributions
1. justify your model structure
1. write model in PyMC3, run inference. 
1. interpret based on posterior distributions
1. (optional) with new information, modify model structure.

# Example 2: Regression

In [ ]:
from sklearn import datasets
import pandas as pd

boston=datasets.load_boston()
boston_data = pd.DataFrame(boston.data)
boston_data.columns = boston['feature_names']
boston_data['MEDV'] = boston.target

In [ ]:
boston_data

## Bayesian Regression

By Default, the prior is  $p(θ) = N(0,10^{12}I)$. This is a very vague prior that will let the data speak for themselves.

In [ ]:
with pm.Model() as regression_model:
    pm.glm.GLM.from_formula('MEDV ~ CRIM+ZN+INDUS+CHAS+NOX+RM+AGE+DIS+RAD+TAX+PTRATIO+B+LSTAT', 
                            boston_data, family=pm.glm.families.Normal())
    normal_trace = pm.sample(3000, chains = 2, cores=2)

In [ ]:
# do not worry about it in Colab
pm.model_to_graphviz(regression_model)

In [ ]:
pm.plot_posterior(normal_trace, hdi_prob=0.95)

In [ ]:
pm.summary(normal_trace)

## Compare with Sklearn

In [ ]:
from sklearn.linear_model import LinearRegression
y = boston_data["MEDV"]
X = boston_data.drop(["MEDV"], axis=1)
model_sk = LinearRegression()
model_sk.fit(X, y)
pd.DataFrame(model_sk.coef_, X.columns)

## Bayesian Ridge and Lasso

The point estimates (*) of the above Bayesian regression with a more informative Gaussian prior is equivalent to a Ridge regression (L2 regularization). If we use a Laplace prior, they are equivalent to a Lasso regression (L1 regularization). 

(*) To be exact, the point estimate from the Bayesian regression is the maximum a posteriori (MAP) estimate, which is the mode of the posterior (highest point in the density function), not the mean. 

See reading *Prior and Regularization* for a proof. 

### Bayesian Ridge

In [ ]:
with pm.Model() as model_ridge:
    # Define priors for intercept and regression coefficients.
    priors = {'Intercept': pm.Normal.dist(mu=0, sigma=5000),
              'Regressor': pm.Normal.dist(mu=0, sigma=1)
    }
    pm.glm.GLM.from_formula('MEDV ~ CRIM+ZN+INDUS+CHAS+NOX+RM+AGE+DIS+RAD+TAX+PTRATIO+B+LSTAT', 
                            boston_data, family=pm.glm.families.Normal(), priors=priors)
    map_estimates = pm.find_MAP()
    # No need the full posterior distribution
#     normal_trace = pm.sample(500, chains = 2, cores=2)

In [ ]:
map_estimates

Equivalent Ridge regression

In [ ]:
from sklearn.linear_model import Ridge
model_l2 = Ridge(alpha=22.85) # Read the reading and find out why we use 22.85 
model_l2.fit(X, y)
pd.DataFrame(model_l2.coef_, X.columns)

### Bayesian Lasso

In [ ]:
from scipy.stats import laplace
x= np.arange(-2,2,0.01)
plt.plot(x, laplace.pdf(x, 0, 0.2))
plt.show()

In [ ]:
with pm.Model() as model_lasso:
    # Define priors for intercept and regression coefficients.
    priors = {'Intercept': pm.Normal.dist(mu=0, sigma=5000),
              'Regressor': pm.Laplace.dist(mu=0, b=0.01)
    }
    pm.glm.GLM.from_formula('MEDV ~ CRIM+ZN+INDUS+CHAS+NOX+RM+AGE+DIS+RAD+TAX+PTRATIO+B+LSTAT', 
                            boston_data, family=pm.glm.families.Normal(), priors=priors)
    map_estimates = pm.find_MAP()
    # No need the full posterior distribution
#     normal_trace = pm.sample(500, chains = 2, cores=2)

In [ ]:
for k, v in map_estimates.items():
    print("{}: {}".format(k, np.round(v,3)))

# Example 3: Hierarchical Linear Regression (Mixed Effects Models)

Gelman et al.’s (2007) radon dataset is a classic for hierarchical modeling. In this dataset the amount of the radioactive gas radon has been measured among different households in all counties of several states. Radon gas is known to be the highest cause of lung cancer in non-smokers. It is believed to be more strongly present in households containing a basement and to differ in amount present among types of soil. Here we’ll investigate this differences and try to make predictions of radonlevels (y) in different counties based on the county itself and the presence of a basement (x). In this example we’ll look at Minnesota, a state that contains 85 counties in which different measurements are taken, ranging from 2 to 116 measurements per county.

This is similar to the random effect model in experimental design.


In [ ]:
import pandas as pd
import theano

data = pd.read_csv(pm.get_data('radon.csv'))
data['log_radon'] = data['log_radon'].astype(theano.config.floatX)
county_names = data.county.unique()
county_idx = data.county_code.values

n_counties = len(data.county.unique())

In [ ]:
data[['log_radon', 'floor', 'county']]

In [ ]:
with pm.Model() as hierarchical_model:
    # Hyperpriors for group nodes
    mu_a = pm.Normal('mu_a', mu=0., sigma=100)
    sigma_a = pm.HalfNormal('sigma_a', 5.)
    mu_b = pm.Normal('mu_b', mu=0., sigma=100)
    sigma_b = pm.HalfNormal('sigma_b', 5.)

    # Intercept for each county, distributed around group mean mu_a.
    # Here we plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    a = pm.Normal('a', mu=mu_a, sigma=sigma_a, shape=n_counties)
    # Intercept for each county, distributed around group mean mu_a
    b = pm.Normal('b', mu=mu_b, sigma=sigma_b, shape=n_counties)

    # Model error
    eps = pm.HalfCauchy('eps', 5.)

    radon_est = a[county_idx] + b[county_idx]*data.floor.values

    # Data likelihood
    radon_like = pm.Normal('randon', mu=radon_est,
                           sigma=eps, observed=data.log_radon)

In [ ]:
# do not worry about it in Colab
pm.model_to_graphviz(hierarchical_model)

In [ ]:
with hierarchical_model:
    hierarchical_trace = pm.sample(2000)

In [ ]:
pm.traceplot(hierarchical_trace,
             var_names=['mu_a', 'mu_b',
                        'sigma_a', 'sigma_b',
                        'eps']);

In [ ]:
pm.summary(hierarchical_trace)

See full example: https://docs.pymc.io/notebooks/GLM-hierarchical.html